# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from lsst_efd_client import EfdClient

In [ ]:
FATABLE = M1M3FATable.FATABLE
client = EfdClient('usdf_efd')

In [ ]:
start = Time("2023-07-11T18:53:30", scale='utc') 
end = Time("2023-07-11T18:58:00", scale='utc') 
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)
monitor_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointMonitorData", "*", start, end)
ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
azs = az['actualPosition'].values
dAz = azs[-1] - azs[0]
els = el['actualPosition'].values
dEl = els[-1] - els[0]
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
print(start.isot, end.isot, timestamp)

In [ ]:
%matplotlib inline
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position changes during slew: {timestamp}\nDeltaAz = {dAz:.3f}, DeltaEl = {dEl:.3f}", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        
        smoothed_ims_data = ims_data[plots[plot_counter]].rolling(10).mean()
        smoothed_ims_data = smoothed_ims_data.dropna()
        smoothed_ims_data -= smoothed_ims_data[0]
        smoothed_hardpoint_data = hardpoint_data[plots[plot_counter]].rolling(10).mean()
        
        if plot_counter > 2:
            smoothed_ims_data *= 1E6
            smoothed_hardpoint_data *= 1E6
            unit = 'microDeg'
        else:
            smoothed_ims_data *= 1E6
            smoothed_hardpoint_data *= 1E6
            unit = 'um'
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'HP Displacement({unit})')
        axs[i][j].yaxis.label.set_color('blue')
        ax = axs[i][j].twinx()
        ax.set_ylabel(f'IMS ({unit})')
        ax.yaxis.label.set_color('red')
        initial_pos = np.median(smoothed_ims_data.values[0:100])
        final_pos = np.median(smoothed_ims_data.values[-100:-1])
        pos_delta = initial_pos - final_pos
        axs[i][j].set_title(plots[plot_counter]+f"\n Position Delta = {pos_delta:.1f} {unit}")

        smoothed_hardpoint_data.plot(ax=axs[i][j], color='blue', label='Hardpoint')
        smoothed_ims_data.plot(ax=ax, color='red', label='IMS')
        plot_counter += 1
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Data_{timestamp}.png")

## Why is displacementLVDT1 flatline here, but displacementLVDT4 flatline in the raise/lower plots?  These things need follow-up.
### Repeat this for raise/lower
### Implement the matrix to convert displacementLVDTx to XYZ positions and rotations.
### Check this with hardpoint displacements and hardpoint positions.

In [ ]:
# Need to convert displacementLVDT values into xyz.

%matplotlib inline
plots = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position changes during slew: {timestamp}\nDeltaAz = {dAz:.3f}, DeltaEl = {dEl:.3f}", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        
        smoothed_ims_data = ims_data[plots[plot_counter]].rolling(10).mean()
        smoothed_ims_data = smoothed_ims_data.dropna()
        smoothed_ims_data -= smoothed_ims_data[0]
        smoothed_monitor_data = monitor_data[f"displacementLVDT{plot_counter}"].rolling(1000).mean()
        
        if plot_counter > 2:
            smoothed_ims_data *= 1E6
            smoothed_monitor_data *= 6.5E3
            unit = 'microDeg'
        else:
            smoothed_ims_data *= 1E6
            smoothed_monitor_data *= 6.5E3
            unit = 'um'
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'displacementLVDT{plot_counter}({unit})')
        axs[i][j].yaxis.label.set_color('blue')
        ax = axs[i][j].twinx()
        ax.set_ylabel(f'IMS ({unit})')
        ax.yaxis.label.set_color('red')
        initial_pos = np.median(smoothed_ims_data.values[0:100])
        final_pos = np.median(smoothed_ims_data.values[-100:-1])
        pos_delta = initial_pos - final_pos
        axs[i][j].set_title(plots[plot_counter]+f"\n Position Delta = {pos_delta:.1f} {unit}")

        smoothed_monitor_data.plot(ax=axs[i][j], color='blue', label='Hardpoint')
        smoothed_ims_data.plot(ax=ax, color='red', label='IMS')
        plot_counter += 1
#plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Data_{timestamp}.png")

In [ ]:

fig, axs = plt.subplots(3,1,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Mirror position measurements: {timestamp}", fontsize=16)

smoothed_ims_data = ims_data['zPosition'].rolling(10).mean()
smoothed_ims_data = smoothed_ims_data.dropna()
smoothed_ims_data -= smoothed_ims_data[0]
smoothed_hardpoint_data = hardpoint_data['zPosition'].rolling(10).mean()
smoothed_lvdt_data = monitor_data['displacementLVDT4'].rolling(1000).mean()

axs[0].set_title('IMS zPosition')
axs[0].set_ylabel(f'zPosition (um)')
axs[0].yaxis.label.set_color('blue')
ax = axs[0].twinx()
ax.set_ylabel(f'Elevation(degrees)')
ax.yaxis.label.set_color('red')
smoothed_ims_data.plot(ax=axs[0], color='blue', label='IMS')
el['actualPosition'].plot(ax=ax, color='red', label='Elevation')

axs[1].set_title('Hardpoint encoder')
axs[1].set_ylabel(f'zPosition (um)')
axs[1].yaxis.label.set_color('blue')
ax = axs[1].twinx()
ax.set_ylabel(f'Elevation(degrees)')
ax.yaxis.label.set_color('red')
smoothed_hardpoint_data.plot(ax=axs[1], color='blue', label='Hardpoint')
el['actualPosition'].plot(ax=ax, color='red', label='Elevation')

axs[2].set_title('LVDT 4')
axs[2].set_ylabel(f'Position (?)')
axs[2].yaxis.label.set_color('blue')
ax = axs[2].twinx()
ax.set_ylabel(f'Elevation(degrees)')
ax.yaxis.label.set_color('red')
smoothed_lvdt_data.plot(ax=axs[2], color='blue', label='Hardpoint')
el['actualPosition'].plot(ax=ax, color='red', label='Elevation')

plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Position_Data_Sources_2_{timestamp}.png")

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=1.0)
plt.suptitle(f"Hardpoint forces during slew: {timestamp}\nDeltaAz = {dAz:.3f}, DeltaEl = {dEl:.3f}", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        name = f"measuredForce{plot_counter}"
        axs[i][j].set_ylabel(f'Force (N)')
        initial_force = np.median(hardpoint_data[name].values[0:100])
        final_force = np.median(hardpoint_data[name].values[-100:-1])
        force_delta = initial_force - final_force
        axs[i][j].set_title(f"Hardpoint {name}\n Force Delta = {force_delta:.1f} N")
        hardpoint_data[name].plot(ax=axs[i][j])
        plot_counter += 1
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Forces_{timestamp}.png")